Dependencies (oh damn I spent 5 hours on this cell)

In [1]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

In [2]:
import os

Let's see what's what

In [7]:
environment_name = "CartPole-v1"
# render_mode = 'human' allows us to see the game for visual validation
env = gym.make(environment_name, render_mode='human')

In [37]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset() # initial set of observations (see below)
    terminated = False
    score = 0

    while not terminated:
        env.render() # for visual
        action = env.action_space.sample() # random action
        observation, reward, terminated, truncated, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:19.0
Episode:2 Score:20.0
Episode:3 Score:11.0
Episode:4 Score:12.0
Episode:5 Score:18.0


In [40]:
env.reset() #initial set of observations

(array([-0.02660681, -0.02088486, -0.02677388,  0.01586788], dtype=float32),
 {})

In [41]:
env.action_space.sample() # random action

0

In [42]:
env.action_space # action space (in our case 2 actions)

Discrete(2)

In [43]:
env.observation_space # observation_space

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

In [44]:
env.observation_space.sample() # random observation (position, angle, velocity, angular velocity)

array([-3.6769652e+00,  2.1461031e+38, -2.3860078e-01,  1.2965613e+38],
      dtype=float32)

In [45]:
env.step(1) # next set of observation after action, reward, ...

(array([-0.02702451,  0.17461063, -0.02645652, -0.2851408 ], dtype=float32),
 1.0,
 False,
 False,
 {})


Training!

In [4]:
log_path = os.path.join('Training', 'Logs')

In [5]:
log_path

'Training\\Logs'

In [8]:
env = gym.make(environment_name, render_mode=None)
env = DummyVecEnv([lambda: env]) # This is a wrapper provided by the stable_baselines3 library. It is used to handle multiple environments simultaneously in a vectorized manner. The vectorized environment allows for parallel execution of multiple environments, which is useful for efficient batch processing during training.
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path) # see more theoretical notebooks for details

Using cpu device


In [9]:
model.learn(total_timesteps=20000)  # see more theoretical notebooks for details

Logging to Training\Logs\PPO_1
-----------------------------
| time/              |      |
|    fps             | 1241 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 797          |
|    iterations           | 2            |
|    time_elapsed         | 5            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0079371445 |
|    clip_fraction        | 0.0795       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.687       |
|    explained_variance   | -0.0034      |
|    learning_rate        | 0.0003       |
|    loss                 | 6.8          |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.0121      |
|    value_loss           | 44.8         |
----------------------------

In [10]:
model

Save the model

In [11]:
PPO_path = os.path.join('Training', 'Saved Models', 'PPO_model')

In [12]:
PPO_path

'Training\\Saved Models\\PPO_model'

In [13]:
model.save(PPO_path) # save the model

In [14]:
del model # delete it

In [15]:
model = PPO.load(PPO_path, env=env) # reloading

In [16]:
model

Let's see how it performs

In [24]:
evaluate_policy(model, env, n_eval_episodes=10, render=False) # perfect score with 0 standard deviation

(500.0, 0.0)

In [25]:
env.close()

In [26]:
env = gym.make(environment_name, render_mode='human')

In [31]:
episodes = 5
for episode in range(1, episodes+1):
    state, _ = env.reset() # initial set of observations (see below)
    terminated = False
    score = 0

    while not terminated:
        env.render() # for visual
        action, _ = model.predict(state) # model prediction
        state, reward, terminated, truncated, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:94.0
Episode:2 Score:726.0
Episode:3 Score:406.0
Episode:4 Score:875.0
Episode:5 Score:185.0


Tenserboard

In [32]:
training_log_path = os.path.join(log_path, 'PPO_1')

In [34]:
training_log_path

'Training\\Logs\\PPO_1'

In [33]:
# !tensorboard --logdir={training_log_path} in the terminal

^C


Callback (making threshold, here made for early stopping)

In [35]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [37]:
save_path = os.path.join('Training', 'Saved Models')
log_path = os.path.join('Training', 'Logs')

In [55]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=190, verbose=1) # Stop the training once a threshold in episodic reward (mean episode reward over the evaluations) has been reached (i.e., when the model is good enough). It must be used with the EvalCallback and use the event triggered by a new best model. 
# is higher than 190
eval_callback = EvalCallback(env, 
                             callback_on_new_best=stop_callback, 
                             eval_freq=10000, 
                             best_model_save_path=save_path, 
                             verbose=1, render=False)  # Evaluate periodically the performance of an agent, using a separate test environment. It will save the best model if best_model_save_path folder is specified and save the evaluations results in a NumPy archive (evaluations.npz) if log_path folder is specified.

In [56]:
env = gym.make(environment_name, render_mode=None)
env = DummyVecEnv([lambda: env])

In [57]:
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cpu device


In [58]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training\Logs\PPO_6
-----------------------------
| time/              |      |
|    fps             | 1291 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 745         |
|    iterations           | 2           |
|    time_elapsed         | 5           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008016864 |
|    clip_fraction        | 0.0952      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | 0.00245     |
|    learning_rate        | 0.0003      |
|    loss                 | 8.35        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0157     |
|    value_loss           | 54          |
-----------------------------------------
---

In [59]:
evaluate_policy(model, env, n_eval_episodes=10, render=False)

(291.6, 124.81522343047742)

What if we want test the other architecture?

In [62]:
net_arch=dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128]) # new architecture parameteres

In [63]:
model = PPO('MlpPolicy', env, verbose = 1, policy_kwargs={'net_arch': net_arch})

Using cpu device


In [64]:
model.learn(total_timesteps=20000, callback=eval_callback)

-----------------------------
| time/              |      |
|    fps             | 791  |
|    iterations      | 1    |
|    time_elapsed    | 2    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 377         |
|    iterations           | 2           |
|    time_elapsed         | 10          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.015082296 |
|    clip_fraction        | 0.236       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.681      |
|    explained_variance   | -0.0037     |
|    learning_rate        | 0.0003      |
|    loss                 | 3.34        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0279     |
|    value_loss           | 19.8        |
-----------------------------------------
----------------------------------

And different training algorythm

In [65]:
from stable_baselines3 import DQN

In [66]:
model = DQN('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cpu device


In [67]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training\Logs\DQN_1
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.956    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 4421     |
|    time_elapsed     | 0        |
|    total_timesteps  | 93       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.908    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 1072     |
|    time_elapsed     | 0        |
|    total_timesteps  | 193      |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.499    |
|    n_updates        | 23       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.862    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 

In [68]:
evaluate_policy(model, env, n_eval_episodes=10, render=False)

(9.0, 0.6324555320336759)